In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

plt.style.use('seaborn')

C:\Users\henry\AppData\Local\Temp\ipykernel_15148\3041625161.py:9: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



In [9]:
df = pd.read_csv('bank.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [13]:
df.value_counts('deposit')

deposit
no     5873
yes    5289
dtype: int64

In [23]:
suscribed_df = df.loc[df["deposit"] == "yes"]
suscribed_df["age"].loc[suscribed_df["job"] == "management"].values

array([42, 56, 29, ..., 62, 25, 30], dtype=int64)

In [3]:
# Drop the Job Occupations that are "Unknown"
df = df.drop(df.loc[df["job"] == "unknown"].index)

# Admin and management are basically the same let's put it under the same categorical value
lst = [df]

for col in lst:
    col.loc[col["job"] == "admin.", "job"] = "management"

In [24]:
df.value_counts('pdays')

pdays
-1      8324
 92      106
 182      89
 91       84
 181      81
        ... 
 387       1
 395       1
 397       1
 399       1
 854       1
Length: 472, dtype: int64

In [4]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,management,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,management,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,management,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [13]:
from sklearn.preprocessing import LabelEncoder
df['deposit'] = LabelEncoder().fit_transform(df['deposit'])
df['poutcome'] = LabelEncoder().fit_transform(df['poutcome'])
df['education'] = LabelEncoder().fit_transform(df['education'])
df['default'] = LabelEncoder().fit_transform(df['default'])
df['housing'] = LabelEncoder().fit_transform(df['housing'])
df['job'] = LabelEncoder().fit_transform(df['job'])
df['loan'] = LabelEncoder().fit_transform(df['loan'])

In [11]:
df = df.select_dtypes(exclude=object)

In [14]:
from sklearn.model_selection import train_test_split

df_data = df.drop(['deposit'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(df_data, df['deposit'], test_size=0.2, stratify=df['loan'])

In [153]:
df['deposit'] = LabelEncoder().fit_transform(df['deposit'])

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
rfc = RandomForestClassifier(n_estimators=50, criterion='gini', oob_score=True, random_state=100, max_depth=11, min_samples_split=50)
rfc = rfc.fit(X_train, Y_train)
score_rfc = rfc.score(X_test, Y_test)

In [24]:
score_rfc

0.8244514106583072

In [25]:
from sklearn.model_selection import cross_val_score
rfc_s = cross_val_score(rfc, df_data, df['deposit'], cv=3)

In [26]:
rfc_s.mean()

0.8022771800851315

In [27]:
rfc.oob_score_

0.8155448538470154

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
# TIME WARNING: These code is used to find the most efficient attribute value, and the total run time will be up to 7 min if your CPU is i7-10750H or lower

param_test2 = {"max_depth":list(range(3,14,2)), "min_samples_split":list(range(50,201,20)), "n_estimators":list(range(1,50,2))}
gsearch2 = GridSearchCV(estimator = rfc,
                        param_grid = param_test2, scoring="roc_auc", cv=3)
gsearch2.fit(X_train,Y_train)
gsearch2.best_params_, gsearch2.best_score_

c:\Users\henry\miniconda3\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:584: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.

c:\Users\henry\miniconda3\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:584: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.

c:\Users\henry\miniconda3\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:584: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.

c:\Users\henry\miniconda3\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:584: UserWarning:

Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.

c:\Users\henry\miniconda3\envs\ds\lib\site-packages\sklearn\ensemble\_forest.py:584: UserWarning:

Some inputs do not have O

({'max_depth': 11, 'min_samples_split': 50, 'n_estimators': 49},
 0.8892892746973208)

In [31]:
rfc.feature_importances_

array([0.06971243, 0.01791055, 0.01425057, 0.00085489, 0.0554161 ,
       0.05941505, 0.01185701, 0.03129609, 0.55620591, 0.02211561,
       0.06326582, 0.03283753, 0.06486244])

In [32]:
# age balance housing duration pdays poutcome

,age,job,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,poutcome,deposit
0,59,0,1,0,2343,1,0,5,1042,1,-1,0,3,1
1,56,0,1,0,45,0,0,5,1467,1,-1,0,3,1
2,41,9,1,0,1270,1,0,5,1389,1,-1,0,3,1
3,55,7,1,0,2476,1,0,5,579,1,-1,0,3,1
4,54,0,2,0,184,0,0,5,673,2,-1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,0,0,1,1,0,20,257,1,-1,0,3,0
11158,39,7,1,0,733,0,0,16,83,4,-1,0,3,0
11159,32,9,1,0,29,0,0,19,156,2,-1,0,3,0
11160,43,9,1,0,0,0,1,8,9,2,172,5,0,0


In [33]:
from sklearn.metrics import confusion_matrix
result = rfc.predict(X_test)
C = confusion_matrix(Y_test, result)
C

array([[973, 211],
       [181, 868]], dtype=int64)

In [34]:
result

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [37]:
len(Y_test[Y_test == 0])

1184

In [41]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, Y_train)
dtc.score(X_test, Y_test)

0.7344379758172862